In [12]:
using DynamicPolynomials, LinearAlgebra, SparseArrays

println("***Problem setting***")

n=1000

println("Number of variable: n=",n)
println("====================")

@ncpolyvar x[1:n]# variables

function star_algebra(mom)
    if mom==1
        return mom
    else
        ind=mom.z .>0
        vars=mom.vars[ind]
        exp=mom.z[ind]
        return prod(vars[i]^exp[i] for i in length(exp):-1:1)
    end
end

function generate_random_poly(T::UnitRange{Int64})
    v=reverse(monomials(x[T],2))
    v+=star_algebra.(v)
    v=v./2
    c=2*rand(Float64,length(v)).-1
    return c'*v
end

# unit sphere constraint
u=15
p=floor(Int64,n/u)+1

I=Vector{UnitRange{Int64}}(undef,p)
I[1]=1:u
I[2:p-1]=[u*(j-1):u*j for j in 2:p-1]
I[p]=u*(p-1):n

# random quadratic objective function f
f=sum(generate_random_poly(I[j]) for j in 1:p)

g=-(x.^2).+(1/u)

m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=ceil(Int64, n/7)

r=floor(Int64,l/p)
W=[(j-1)*r+1:j*r for j in 1:p-1]
append!(W,[(p-1)*r+1:l])

h=Vector{Polynomial{false,Float64}}(undef,l)

randx=2*rand(n).-1
randx=randx./sqrt(u)

for j in 1:p
    for i in W[j]
        h[i]=generate_random_poly(I[j])
        h[i]-=h[i](x => randx) #make constraints feasible
    end
end

l=length(h)
println("Number of equality constraints: l=",l)
println("====================")

***Problem setting***
Number of variable: n=1000
Number of inequality constraints: m=1000
Number of equality constraints: l=143


In [13]:
include("../src/ctpNCPOP.jl")
using .ctpNCPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpNCPOP.get_info(x,f,g,h);

In [14]:
k=2

println("Relaxed order: k=",k)
println("====================")

t=1

println("Sparse order: t=",t)
println("====================")

Relaxed order: k=2
Sparse order: t=1


In [ ]:
include("../src/ctpNCPOP.jl")
using .ctpNCPOP

opt_val1=ctpNCPOP.POP_mix_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,t,
    maxit=Int64(1e10),tol=5e-3,use_eqcons_to_get_constant_trace=false,check_tol_each_iter=true)

------------------------------------------------------
The clique sizes of varibles:
[16, 15, 11]
[65, 1, 1]
------------------------------------------------------
  Number of cliques: p=67
  Largest clique size: u=16


  Largest constant trace: a=3.204444444444444
  Number of blocks: omega=2266
  Number of equality consraints: zeta=181006
  Size of the largest block: s^max=257
Modeling time:
131.919565 seconds (1.18 G allocations: 148.159 GiB, 12.90% gc time)
iter=1   val=-613.584187751235   gap=1.8498171607310047e-16   feas=22.075947096897544
iter=2   val=-511.9467265153123   gap=0.09052768794835982   feas=18.644517799791178
iter=4   val=-356.1217691047821   gap=0.10477497354136212   feas=9.947926800117276
iter=8   val=-267.0984720877958   gap=0.14434935300759047   feas=4.825927628129485
iter=16   val=-203.44272931505682   gap=0.25897100195452605   feas=2.3959110718862275
iter=32   val=-203.66845663280776   gap=0.11896362591516409   feas=1.5953534048391402
iter=64   val=-189.86782097514464   gap=0.07311962470773213   feas=0.9846846704825888
iter=128   val=-167.98460240665122   gap=0.08546421962791344   feas=0.5186677885983717
iter=256   val=-159.2491636299329   gap=0.04686997963075526   feas=0.25332

In [11]:
using NCTSSOS

@time begin

j=1
opt,data=cs_nctssos_first(Vector{Vector{Vector{UInt16}}}([[supp_f];supp_g;supp_h]),[[coe_f];coe_g;coe_h],
        n,k,[dg;dh],numeq=l,minimize=true,obj="eigen",TS="block",QUIET=false);
while j <= t-1
    opt,data=cs_nctssos_higher!(data,TS="MD")
    j+=1
end
    
end

------------------------------------------------------
The clique sizes of varibles:
[11, 10]
[99, 1]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 742161          
  Cones                  : 0               
  Scalar variables       : 1144            
  Matrix variables       : 1343            
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.14            
Lin. dep.  - number                 : 0               
Presolve terminated. Tim

In [6]:
#using ctpPOP

#optval=ctpPOP.POP_NLP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)